***
### <span style='color:green'> ME Algorithm  &emsp;&emsp; Feb, 2024 </span>
### <span style='color:Blue'> Phase 3 </span>
### <p> Yan-Bin Chen (陳彥賓) &emsp; yanbin@stat.sinica.edu.tw </p>
### <p> Institute of Statistical Science, Academia Sinica, Taipei, Taiwan.</p>  
---

# Input

In [1]:
import pickle
import numpy as np
import collections
import pandas
import itertools
from itertools import chain
import random 
import time
import datetime

In [2]:
# Input
PATH1='../data/seedinds.txt'
PATH2='../data/bilabels.txt'
PATH3='../data/seedinds_neighborregions.txt'
PATH4='../data/ResNet18_PlantDisease_45K_Spec200_sampling.csv'
PATH5='../data/ResNet18_PlantDisease_45K_Values_sampling.csv'


#===  parameters ========================================================
TRIALS    = 1
timestr   = ''
REG_COLUMN = "Spec200"
RAW_2D_DATA = False

if RAW_2D_DATA: # 2D
    from CNN_Modules import ME_CNN
else: # 1D
    from CNN_Modules_1D import ME_CNN


# (1)PATH1
df1 = pandas.read_csv(PATH1, header=None, delimiter = "\t")
region = df1.to_numpy().T[0]
NUM_region = len(region)

# (2)PATH2
df2 = pandas.read_csv(PATH2, header=None, delimiter = "\t")
cen = df2.to_numpy()

print('region:', region)
print('region amount:', len(region))


# (4)PATH4. Have to be here. The following neighboring process needs this information
df4 = pandas.read_csv(PATH4)
display(df4.head())
#all_region_index  = df.iloc[:,REGION_INDEX_LOC].to_numpy().astype(int)
all_region_index  = df4[REG_COLUMN].to_numpy().astype(int)
print("all_region_index\n",len(all_region_index), all_region_index)


# (4)PATH4 reset label
label=df4['Label'].value_counts(sort=False).keys()
_=[df4['Label'].replace(to_replace=label[i], value=i, inplace=True) for i in range(len(label))]
display(df4['Label'].value_counts(sort=False))
test_label_answer = df4["Label"].to_numpy()

region: [ 26 182 169 124  29   1  57  76  53  52 183  59  24 164]
region amount: 14


,X1,X2,X3,Class,Label,Spec200
0,1.230895,-11.513212,18.781408,Cherry,2,177
1,3.190353,-14.243707,14.359673,Cherry,2,177
2,1.832576,-12.109021,15.785449,Cherry,2,177
3,1.880559,-12.113366,15.720321,Cherry,2,177
4,5.759654,-13.821998,15.943708,Cherry,2,177


all_region_index
 13076 [177 177 177 ...  53  29 144]


0     147
8    3840
1     746
2     434
3    4375
4    2708
5     826
Name: Label, dtype: int64

In [3]:
# (5)PATH5     convert the embedded data into the pickle file
df5 = pandas.read_csv(PATH5)
test_array = df5.to_numpy()


#save
PATH5='../data/embedded_data.pickle'   #replace original PATH 5
with open(PATH5, 'wb') as f:
    pickle.dump([test_array, test_label_answer], f)
print(test_label_answer, "\n", test_array)

[2 2 2 ... 8 8 8] 
 [[-1.7  -0.2  -1.07 ...  0.46  3.43  0.45]
 [-1.58 -2.04 -1.33 ...  0.03  1.79  0.56]
 [-1.55 -0.9  -1.15 ...  0.51  2.4   1.41]
 ...
 [ 1.39 -0.22 -1.5  ...  1.48  1.93  0.72]
 [ 0.34 -1.75 -3.64 ... -0.76  3.19  1.24]
 [ 0.79 -0.65 -1.39 ... -1.37  4.82 -0.03]]


# Get neighbors

In [4]:
if (PATH3):
    df = pandas.read_csv(PATH3, delim_whitespace=' ', header=None,  index_col=0)
    neighbors = df.to_numpy()
    NUM_NEI=df.shape[1]
    
    print('neighbor amount: ',len(list(chain.from_iterable(neighbors))))
    print(neighbors)

# filter out duplicated ones
    test_list=list(chain.from_iterable(neighbors))
    res2=[]
    [res2.append(n) for n, i in enumerate(test_list) if i in test_list[:n]]

    res2.reverse()
    neighbors=neighbors.tolist()
    [neighbors[x//NUM_NEI].pop(x%NUM_NEI) for x in res2]
    print('neighbor amount: ',len(list(chain.from_iterable(neighbors))))
    display(neighbors)

neighbor amount:  70
[[ 3  4  5  6  8]
 [ 3  4  5  6  8]
 [ 3  4  5  6  8]
 [ 3  4  5  6  8]
 [ 3  4  5  6  8]
 [11 12 14 15 17]
 [ 3  4  5  6  8]
 [ 3  4  5  6  8]
 [ 3  4  5  6  8]
 [ 3  4  5  6  8]
 [ 3  4  5  6  8]
 [ 3  4  5  6  8]
 [ 3  4  5  6  8]
 [ 3  4  5  6  8]]
neighbor amount:  10


[[3, 4, 5, 6, 8],
 [],
 [],
 [],
 [],
 [11, 12, 14, 15, 17],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [5]:
# combine seed regions and neighbors
if (PATH3):
    reg_nei=[]
    for i in range(len(neighbors)):
        a=[region[i]]
        b=neighbors[i]
        if len(b):
            c=list(np.concatenate((a,b),axis=0))
        else:
            c=a.copy()
        reg_nei.append(c)
else:
    reg_nei=region.copy()

reg_nei

[[26, 3, 4, 5, 6, 8],
 [182],
 [169],
 [124],
 [29],
 [1, 11, 12, 14, 15, 17],
 [57],
 [76],
 [53],
 [52],
 [183],
 [59],
 [24],
 [164]]

In [6]:
# collect images from target region and neighboring regions
# input : region, NUM_region, cen, all_region_index, neighbors
# output: region_image: to save image indices corresponding to seed regions.
#         region_answer: to save answer

region_image_before=[]
region_image=[]
region_image_pure=[]
for i in range(NUM_region):
    
    
    #(1)neighbor  nei
    if (PATH3):
        addr_nei=[]
        for j in range(len(neighbors[i])):
            addr_nei=addr_nei+list(np.where(all_region_index==neighbors[i][j])[0])
            #check whether it has duplicates
            if (len(addr_nei) != len(set(addr_nei))):
                print("neighbor duplicate at i=",i,"j=",j)
                addr_nei=list(set(addr_nei))

    #(2)original
    addr=list(np.where(  (all_region_index==region[i])  &  (cen.T[1]==1)  )[0])
    if (PATH3):
        addr=addr+addr_nei
    #check whether it has duplicates
    if (len(addr) != len(set(addr))):
        print("** original duplicate at i=",i,";region",region[i],";duplicate size",len(addr)-len(set(addr)))
        addr=list(set(addr))
    region_image.append(addr)

    #(3)original before centroid (this is only for check, rather than for main codes)
    addr_before=list(np.where(all_region_index==region[i])[0])
    if (PATH3):
        addr_before=addr_before+addr_nei
    #check whether it has duplicates
    if (len(addr_before) != len(set(addr_before))):
        print("** same duplicate situation")
        addr_before=list(set(addr_before))
    region_image_before.append(addr_before)

    #(4)pure
    addr_pure=list(np.where(  (all_region_index==region[i])  &  (cen.T[1]==1)  )[0])
    region_image_pure.append(addr_pure)


with open('../data/' + timestr + 'region_for_phase5.pickle', 'wb') as f:
    pickle.dump([region, reg_nei, region_image_pure, region_image,], f)

In [7]:
# ==== test_array ====
with open(PATH5, 'rb') as f:
    test_array, test_label_answer = pickle.load(f)
print("test_array:",np.shape(test_array))


# 1213 add auto judge /255
# 20240319
if RAW_2D_DATA: # 2D
    print("")
else: # 1D
    test_array = np.expand_dims(test_array, axis = -1)
test_array /= 255
print("training size:",len(np.array(list(chain.from_iterable(region_image)))))

test_array: (13076, 1000)
training size: 3601


# CNN original

In [8]:
# ==== CNN Original =====
ROUND_start = time.time()

for n in range(5):
    region=region_image.copy()
    region=list(region)
    selected_region = list(range(NUM_region))

    p1=0
    p2=0

    print("n, p1, p2", n, p1, p2)


    # ===== one CNN =============
    NUM_CLASSES = len(selected_region)

    # input image and label
    Input_img     = []
    Input_img_len = []
    for c,sel in enumerate(selected_region, start=0):
        Input_img = Input_img + list(region[sel])
        Input_img_len.append(len(region[sel]))


    # fill up training array
        
    # 20240319
    if RAW_2D_DATA: # 2D
        W           = np.shape(test_array[0])[0]
        H           = np.shape(test_array[0])[1]
        train_array = np.zeros((len(Input_img), W, H), dtype=float)
        for i in range (len(Input_img)):
            train_array[i] = test_array[Input_img[i]].reshape(W,H)
    else: # 1D
        W           = np.shape(test_array[0])[0]
        train_array = np.zeros((len(Input_img), W), dtype=float)
        for i in range (len(Input_img)):
            train_array[i] = test_array[Input_img[i]].reshape(W)
        


    train_array = np.expand_dims(train_array, axis = -1)


    # fill up the training label to each training image
    current_train_label = np.zeros(len(train_array), dtype=int)
    accum_base=0
    for label in range(1,NUM_CLASSES):
        pre_cursor = Input_img_len[label-1]
        accum_base = accum_base + pre_cursor
        current_train_label[accum_base:] = label

    # CNN
    #===============================================
    one_predicted_results  = np.zeros((TRIALS, len(test_label_answer)), dtype=int)
    one_predict_percentage = np.zeros((TRIALS, len(test_label_answer), NUM_CLASSES), dtype=float)    
    model_history = np.zeros(TRIALS, dtype=list)

    for r in range(TRIALS):
        one_predicted_results[r], one_predict_percentage[r], model_history[r] = ME_CNN(
                x_train     = train_array,
                train_label = current_train_label,
                test_array  = test_array,
                true_answer = test_label_answer,
                Num_Classes = NUM_CLASSES
                )
        print(type(model_history))


        # ===== delete CNN tensors =====
        from keras import backend as K
        K.clear_session()
        import gc
        gc.collect()

        print("One CNN, r: ",r)
        ROUND_duration = time.time() - ROUND_start
        print("Computing Time: ", str(datetime.timedelta(seconds=ROUND_duration)))

    # === save to file ===
    savefile_path = './' + str(timestr) + '(classes=' + str(NUM_CLASSES)+')_n0_R' + str(p1) + '+R'+ str(p2) +'_trial' + str(n)+'.pickle'  #extra_original
    with open(savefile_path, 'wb') as f:
        pickle.dump([Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history], f)

n, p1, p2 0 0 0


Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1000, 6)           36        
_________________________________________________________________
average_pooling1d_1 (Average (None, 500, 6)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 496, 16)           496       
_________________________________________________________________
average_pooling1d_2 (Average (None, 248, 16)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 244, 120)          9720      
_________________________________________________________________
flatten_1 (Flatten)          (None, 29280)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 84)               

3240/3240 [==============================] - 0s 87us/step - loss: 1.7042e-04 - accuracy: 1.0000 - val_loss: 8.3451e-04 - val_accuracy: 1.0000
Epoch 47/80
3240/3240 [==============================] - 0s 87us/step - loss: 1.6752e-04 - accuracy: 1.0000 - val_loss: 0.0014 - val_accuracy: 1.0000
Epoch 48/80
3240/3240 [==============================] - 0s 82us/step - loss: 1.7456e-04 - accuracy: 1.0000 - val_loss: 0.0011 - val_accuracy: 1.0000
Epoch 49/80
3240/3240 [==============================] - 0s 82us/step - loss: 1.4410e-04 - accuracy: 1.0000 - val_loss: 8.3959e-04 - val_accuracy: 1.0000
Epoch 50/80
3240/3240 [==============================] - 0s 82us/step - loss: 1.3518e-04 - accuracy: 1.0000 - val_loss: 9.6494e-04 - val_accuracy: 1.0000
Epoch 51/80
3240/3240 [==============================] - 0s 92us/step - loss: 1.3616e-04 - accuracy: 1.0000 - val_loss: 0.0025 - val_accuracy: 1.0000
Epoch 52/80
3240/3240 [==============================] - 0s 87us/step - loss: 1.3779e-04 - accuracy:

Epoch 6/80
3240/3240 [==============================] - 0s 87us/step - loss: 0.0238 - accuracy: 0.9951 - val_loss: 0.0465 - val_accuracy: 0.9861
Epoch 7/80
3240/3240 [==============================] - 0s 85us/step - loss: 0.0222 - accuracy: 0.9941 - val_loss: 0.0167 - val_accuracy: 0.9972
Epoch 8/80
3240/3240 [==============================] - 0s 85us/step - loss: 0.0127 - accuracy: 0.9981 - val_loss: 0.0114 - val_accuracy: 0.9972
Epoch 9/80
3240/3240 [==============================] - 0s 86us/step - loss: 0.0108 - accuracy: 0.9985 - val_loss: 0.0087 - val_accuracy: 0.9972
Epoch 10/80
3240/3240 [==============================] - 0s 86us/step - loss: 0.0099 - accuracy: 0.9985 - val_loss: 0.0141 - val_accuracy: 0.9972
Epoch 11/80
3240/3240 [==============================] - 0s 83us/step - loss: 0.0076 - accuracy: 0.9985 - val_loss: 0.0064 - val_accuracy: 1.0000
Epoch 12/80
3240/3240 [==============================] - 0s 90us/step - loss: 0.0047 - accuracy: 1.0000 - val_loss: 0.0108 - val

Train on 3240 samples, validate on 361 samples
Epoch 1/80
3240/3240 [==============================] - 1s 164us/step - loss: 2.4986 - accuracy: 0.2130 - val_loss: 2.3079 - val_accuracy: 0.3380
Epoch 2/80
3240/3240 [==============================] - 0s 92us/step - loss: 1.8205 - accuracy: 0.5034 - val_loss: 1.1711 - val_accuracy: 0.6537
Epoch 3/80
3240/3240 [==============================] - 0s 87us/step - loss: 0.5314 - accuracy: 0.8855 - val_loss: 0.1599 - val_accuracy: 0.9584
Epoch 4/80
3240/3240 [==============================] - 0s 89us/step - loss: 0.1107 - accuracy: 0.9728 - val_loss: 0.0653 - val_accuracy: 0.9834
Epoch 5/80
3240/3240 [==============================] - 0s 84us/step - loss: 0.0480 - accuracy: 0.9873 - val_loss: 0.0370 - val_accuracy: 0.9917
Epoch 6/80
3240/3240 [==============================] - 0s 87us/step - loss: 0.0279 - accuracy: 0.9944 - val_loss: 0.0279 - val_accuracy: 0.9917
Epoch 7/80
3240/3240 [==============================] - 0s 91us/step - loss: 0.018

Epoch 13/80
3240/3240 [==============================] - 0s 87us/step - loss: 0.0058 - accuracy: 0.9991 - val_loss: 0.0073 - val_accuracy: 0.9972
Epoch 14/80
3240/3240 [==============================] - 0s 90us/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0.0089 - val_accuracy: 0.9945
Epoch 15/80
3240/3240 [==============================] - 0s 87us/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.0068 - val_accuracy: 0.9972
Epoch 16/80
3240/3240 [==============================] - 0s 89us/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.0073 - val_accuracy: 0.9972
Epoch 17/80
3240/3240 [==============================] - 0s 84us/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.0129 - val_accuracy: 0.9972
Epoch 18/80
3240/3240 [==============================] - 0s 87us/step - loss: 0.0041 - accuracy: 0.9991 - val_loss: 0.0046 - val_accuracy: 1.0000
Epoch 19/80
3240/3240 [==============================] - 0s 90us/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.0044 -

3240/3240 [==============================] - 0s 85us/step - loss: 0.0272 - accuracy: 0.9941 - val_loss: 0.0218 - val_accuracy: 0.9972
Epoch 8/80
3240/3240 [==============================] - 0s 88us/step - loss: 0.0219 - accuracy: 0.9951 - val_loss: 0.0121 - val_accuracy: 0.9972
Epoch 9/80
3240/3240 [==============================] - 0s 86us/step - loss: 0.0140 - accuracy: 0.9981 - val_loss: 0.0084 - val_accuracy: 1.0000
Epoch 10/80
3240/3240 [==============================] - 0s 88us/step - loss: 0.0178 - accuracy: 0.9954 - val_loss: 0.0076 - val_accuracy: 1.0000
Epoch 11/80
3240/3240 [==============================] - 0s 85us/step - loss: 0.0103 - accuracy: 0.9985 - val_loss: 0.0075 - val_accuracy: 1.0000
Epoch 12/80
3240/3240 [==============================] - 0s 89us/step - loss: 0.0096 - accuracy: 0.9991 - val_loss: 0.0164 - val_accuracy: 0.9945
Epoch 13/80
3240/3240 [==============================] - 0s 85us/step - loss: 0.0071 - accuracy: 0.9988 - val_loss: 0.0080 - val_accuracy:

3240/3240 [==============================] - 0s 86us/step - loss: 1.2070e-04 - accuracy: 1.0000 - val_loss: 2.8519e-04 - val_accuracy: 1.0000
Epoch 62/80
3240/3240 [==============================] - 0s 87us/step - loss: 1.1576e-04 - accuracy: 1.0000 - val_loss: 3.0073e-04 - val_accuracy: 1.0000
Epoch 63/80
3240/3240 [==============================] - 0s 86us/step - loss: 1.1234e-04 - accuracy: 1.0000 - val_loss: 3.0132e-04 - val_accuracy: 1.0000
Epoch 64/80
3240/3240 [==============================] - 0s 82us/step - loss: 1.0244e-04 - accuracy: 1.0000 - val_loss: 3.2002e-04 - val_accuracy: 1.0000
Epoch 65/80
3240/3240 [==============================] - 0s 87us/step - loss: 1.1311e-04 - accuracy: 1.0000 - val_loss: 2.5806e-04 - val_accuracy: 1.0000
Epoch 66/80
3240/3240 [==============================] - 0s 86us/step - loss: 1.1279e-04 - accuracy: 1.0000 - val_loss: 2.8862e-04 - val_accuracy: 1.0000
Epoch 67/80
3240/3240 [==============================] - 0s 87us/step - loss: 9.3772e-05

In [9]:
#==== shift label =======
N=5
Original_result=[]
Original_prob=[]
for i in range(N):
    with open('./' + timestr + '(classes=' + str(NUM_region) + ')_n0_R0+R0_trial' + str(i)+ '.pickle', 'rb') as f:
        Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history = pickle.load(f)
    label_B = one_predicted_results[0].copy()
    label_B_prob = one_predict_percentage[0].copy()

    #(1)
    Original_result.append(label_B)

    #(2)
    Original_prob.append(label_B_prob)
print(np.shape(Original_result), np.shape(Original_prob))

# save pickle
with open('./' + timestr + 'results_of_original.pickle', 'wb') as f:
    pickle.dump([Original_result, Original_prob], f)
    
# save mat
from scipy.io import savemat
savemat('./' + timestr + 'results_of_original.mat', {'result_for_original':Original_result, 'prob_for_original':Original_prob})

(5, 13076) (5, 13076, 14)


# CNN combination and removal

In [10]:
#====CNN combination c(20,2)======
comb=[]
for subset in itertools.combinations(range(NUM_region), 2):
    comb.append(subset)
NUM_comb=len(comb)
display(NUM_comb)            

for n in range(NUM_comb+1):
    region=region_image.copy()
    region=list(region)
    selected_region = list(range(NUM_region))

    if (n > 0):
        p1=comb[n-1][0]
        p2=comb[n-1][1]
        region[p1]=region[p1]+region[p2]
        region.pop(p2)
        selected_region.pop(-1)
    else:
        p1=0
        p2=0

    print("n, p1, p2", n, p1, p2)


    # ===== one CNN =============
    NUM_CLASSES = len(selected_region)

    Input_img     = []
    Input_img_len = []
    for c,sel in enumerate(selected_region, start=0):
        Input_img = Input_img + list(region[sel])
        Input_img_len.append(len(region[sel]))

    # 20240319
    if RAW_2D_DATA: # 2D
        W           = np.shape(test_array[0])[0]
        H           = np.shape(test_array[0])[1]
        train_array = np.zeros((len(Input_img), W, H), dtype=float)
        for i in range (len(Input_img)):
            train_array[i] = test_array[Input_img[i]].reshape(W,H)
    else: # 1D
        W           = np.shape(test_array[0])[0]
        train_array = np.zeros((len(Input_img), W), dtype=float)
        for i in range (len(Input_img)):
            train_array[i] = test_array[Input_img[i]].reshape(W)

    train_array = np.expand_dims(train_array, axis = -1)


    # fill up the training label to each training image
    current_train_label = np.zeros(len(train_array), dtype=int)
    accum_base=0
    for label in range(1,NUM_CLASSES):
        pre_cursor = Input_img_len[label-1]
        accum_base = accum_base + pre_cursor
        current_train_label[accum_base:] = label

    # CNN
    #===============================================
    one_predicted_results  = np.zeros((TRIALS, len(test_label_answer)), dtype=int)
    one_predict_percentage = np.zeros((TRIALS, len(test_label_answer), NUM_CLASSES), dtype=float)    
    model_history = np.zeros(TRIALS, dtype=list)

    for r in range(TRIALS):
        one_predicted_results[r], one_predict_percentage[r], model_history[r] = ME_CNN(
                x_train     = train_array,
                train_label = current_train_label,
                test_array  = test_array,
                true_answer = test_label_answer,
                Num_Classes = NUM_CLASSES
                )
        print(type(model_history))


        # ===== delete CNN tensors =====
        from keras import backend as K
        K.clear_session()
        import gc
        gc.collect()

        print("One CNN, r: ",r)
        ROUND_duration = time.time() - ROUND_start
        print("Computing Time: ", str(datetime.timedelta(seconds=ROUND_duration)))

    # === save to file ===
    savefile_path = './' + str(timestr) + '(classes=' + str(NUM_CLASSES)+')_n' + str(n) + '_R' + str(p1) + '+R'+ str(p2) +'.pickle'
    with open(savefile_path, 'wb') as f:
        pickle.dump([Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history], f)



# ==== CNN Removal =====
for n in range(NUM_region):
    region=region_image.copy()   #reset
    selected_region = list(range(NUM_region))
    selected_region.pop(n)

    print("n=", n)


    # ===== one CNN =============
    NUM_CLASSES = len(selected_region)

    # input image and label
    Input_img     = []
    Input_img_len = []
    for c,sel in enumerate(selected_region, start=0):
        Input_img = Input_img + list(region[sel])
        Input_img_len.append(len(region[sel]))


    # 20240319
    if RAW_2D_DATA: # 2D
        W           = np.shape(test_array[0])[0]
        H           = np.shape(test_array[0])[1]
        train_array = np.zeros((len(Input_img), W, H), dtype=float)
        for i in range (len(Input_img)):
            train_array[i] = test_array[Input_img[i]].reshape(W,H)
    else: # 1D
        W           = np.shape(test_array[0])[0]
        train_array = np.zeros((len(Input_img), W), dtype=float)
        for i in range (len(Input_img)):
            train_array[i] = test_array[Input_img[i]].reshape(W)

    train_array = np.expand_dims(train_array, axis = -1)
    
    
    # fill up the training label to each training image
    current_train_label = np.zeros(len(train_array), dtype=int)
    accum_base=0
    for label in range(1,NUM_CLASSES):
        pre_cursor = Input_img_len[label-1]
        accum_base = accum_base + pre_cursor
        current_train_label[accum_base:] = label

    # CNN
    #===============================================
    one_predicted_results  = np.zeros((TRIALS, len(test_label_answer)), dtype=int)
    one_predict_percentage = np.zeros((TRIALS, len(test_label_answer), NUM_CLASSES), dtype=float)    
    model_history = np.zeros(TRIALS, dtype=list)

    for r in range(TRIALS):
        one_predicted_results[r], one_predict_percentage[r], model_history[r] = ME_CNN(
                x_train     = train_array,
                train_label = current_train_label,
                test_array  = test_array,
                true_answer = test_label_answer,
                Num_Classes = NUM_CLASSES
                )
        print(type(model_history))


        # ===== delete CNN tensors =====
        from keras import backend as K
        K.clear_session()
        import gc
        gc.collect()

        print("One CNN, r: ",r)
        ROUND_duration = time.time() - ROUND_start
        print("Computing Time: ", str(datetime.timedelta(seconds=ROUND_duration)))

    # === save to file ===
    savefile_path = './' + str(timestr) + '(classes=' + str(NUM_CLASSES)+')_Remove' + str(n) +'.pickle' 
    with open(savefile_path, 'wb') as f:
        pickle.dump([Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history], f)
  

91

n, p1, p2 0 0 0
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1000, 6)           36        
_________________________________________________________________
average_pooling1d_1 (Average (None, 500, 6)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 496, 16)           496       
_________________________________________________________________
average_pooling1d_2 (Average (None, 248, 16)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 244, 120)          9720      
_________________________________________________________________
flatten_1 (Flatten)          (None, 29280)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 84

[2 2 2 ... 8 4 8]
<class 'numpy.ndarray'>
One CNN, r:  0
Computing Time:  0:02:06.390512
n, p1, p2 1 0 1
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1000, 6)           36        
_________________________________________________________________
average_pooling1d_1 (Average (None, 500, 6)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 496, 16)           496       
_________________________________________________________________
average_pooling1d_2 (Average (None, 248, 16)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 244, 120)          9720      
_________________________________________________________________
flatten_1 (Flatten)          (None, 29280)             0         
_______________

[1 1 1 ... 7 3 7]
<class 'numpy.ndarray'>
One CNN, r:  0
Computing Time:  0:02:21.366871
n, p1, p2 2 0 2
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1000, 6)           36        
_________________________________________________________________
average_pooling1d_1 (Average (None, 500, 6)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 496, 16)           496       
_________________________________________________________________
average_pooling1d_2 (Average (None, 248, 16)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 244, 120)          9720      
_________________________________________________________________
flatten_1 (Flatten)          (None, 29280)             0         
_______________

Epoch 46/80
3240/3240 [==============================] - 0s 87us/step - loss: 2.1265e-04 - accuracy: 1.0000 - val_loss: 0.0021 - val_accuracy: 1.0000
Epoch 47/80
3240/3240 [==============================] - 0s 89us/step - loss: 1.9825e-04 - accuracy: 1.0000 - val_loss: 0.0025 - val_accuracy: 1.0000
Epoch 48/80
3240/3240 [==============================] - 0s 88us/step - loss: 1.9399e-04 - accuracy: 1.0000 - val_loss: 0.0025 - val_accuracy: 1.0000
Epoch 49/80
3240/3240 [==============================] - 0s 87us/step - loss: 1.8483e-04 - accuracy: 1.0000 - val_loss: 0.0023 - val_accuracy: 1.0000
Epoch 50/80
3240/3240 [==============================] - 0s 87us/step - loss: 1.7726e-04 - accuracy: 1.0000 - val_loss: 0.0024 - val_accuracy: 1.0000
Epoch 51/80
3240/3240 [==============================] - 0s 91us/step - loss: 1.7640e-04 - accuracy: 1.0000 - val_loss: 0.0032 - val_accuracy: 1.0000
Epoch 52/80
3240/3240 [==============================] - 0s 87us/step - loss: 1.6538e-04 - accuracy:

...

3081/3081 [==============================] - 0s 89us/step - loss: 7.7983e-04 - accuracy: 1.0000 - val_loss: 0.0022 - val_accuracy: 1.0000
Epoch 33/80
3081/3081 [==============================] - 0s 87us/step - loss: 7.5124e-04 - accuracy: 1.0000 - val_loss: 0.0016 - val_accuracy: 1.0000
Epoch 34/80
3081/3081 [==============================] - 0s 87us/step - loss: 7.2409e-04 - accuracy: 1.0000 - val_loss: 0.0017 - val_accuracy: 1.0000
Epoch 35/80
3081/3081 [==============================] - 0s 87us/step - loss: 8.6387e-04 - accuracy: 1.0000 - val_loss: 0.0027 - val_accuracy: 1.0000
Epoch 36/80
3081/3081 [==============================] - 0s 89us/step - loss: 0.0030 - accuracy: 0.9984 - val_loss: 0.0099 - val_accuracy: 0.9971
Epoch 37/80
3081/3081 [==============================] - 0s 83us/step - loss: 0.0062 - accuracy: 0.9984 - val_loss: 0.0039 - val_accuracy: 1.0000
Epoch 38/80
3081/3081 [==============================] - 0s 90us/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.00

Train on 2790 samples, validate on 310 samples
Epoch 1/80
2790/2790 [==============================] - 1s 224us/step - loss: 2.4008 - accuracy: 0.2326 - val_loss: 2.1488 - val_accuracy: 0.3484
Epoch 2/80
2790/2790 [==============================] - 0s 97us/step - loss: 1.5384 - accuracy: 0.5548 - val_loss: 0.7787 - val_accuracy: 0.8323
Epoch 3/80
2790/2790 [==============================] - 0s 87us/step - loss: 0.3379 - accuracy: 0.9401 - val_loss: 0.1172 - val_accuracy: 0.9774
Epoch 4/80
2790/2790 [==============================] - 0s 91us/step - loss: 0.0783 - accuracy: 0.9835 - val_loss: 0.0407 - val_accuracy: 0.9968
Epoch 5/80
2790/2790 [==============================] - 0s 87us/step - loss: 0.0380 - accuracy: 0.9910 - val_loss: 0.0226 - val_accuracy: 0.9968
Epoch 6/80
2790/2790 [==============================] - 0s 96us/step - loss: 0.0237 - accuracy: 0.9953 - val_loss: 0.0145 - val_accuracy: 1.0000
Epoch 7/80
2790/2790 [==============================] - 0s 86us/step - loss: 0.019

Epoch 56/80
2790/2790 [==============================] - 0s 88us/step - loss: 1.2015e-04 - accuracy: 1.0000 - val_loss: 0.0017 - val_accuracy: 1.0000
Epoch 57/80
2790/2790 [==============================] - 0s 87us/step - loss: 1.2039e-04 - accuracy: 1.0000 - val_loss: 0.0018 - val_accuracy: 1.0000
Epoch 58/80
2790/2790 [==============================] - 0s 85us/step - loss: 1.1132e-04 - accuracy: 1.0000 - val_loss: 0.0015 - val_accuracy: 1.0000
Epoch 59/80
2790/2790 [==============================] - 0s 93us/step - loss: 1.0549e-04 - accuracy: 1.0000 - val_loss: 0.0016 - val_accuracy: 1.0000
Epoch 60/80
2790/2790 [==============================] - 0s 86us/step - loss: 1.0151e-04 - accuracy: 1.0000 - val_loss: 0.0016 - val_accuracy: 1.0000
Epoch 61/80
2790/2790 [==============================] - 0s 92us/step - loss: 9.8813e-05 - accuracy: 1.0000 - val_loss: 0.0017 - val_accuracy: 1.0000
Epoch 62/80
2790/2790 [==============================] - 0s 87us/step - loss: 9.9135e-05 - accuracy:

Epoch 17/80
3076/3076 [==============================] - 0s 88us/step - loss: 0.0042 - accuracy: 0.9993 - val_loss: 0.0145 - val_accuracy: 0.9942
Epoch 18/80
3076/3076 [==============================] - 0s 88us/step - loss: 0.0032 - accuracy: 1.0000 - val_loss: 0.0144 - val_accuracy: 0.9942
Epoch 19/80
3076/3076 [==============================] - 0s 92us/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.0171 - val_accuracy: 0.9942
Epoch 20/80
3076/3076 [==============================] - 0s 86us/step - loss: 0.0026 - accuracy: 0.9997 - val_loss: 0.0148 - val_accuracy: 0.9942
Epoch 21/80
3076/3076 [==============================] - 0s 86us/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.0141 - val_accuracy: 0.9942
Epoch 22/80
3076/3076 [==============================] - 0s 92us/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.0149 - val_accuracy: 0.9942
Epoch 23/80
3076/3076 [==============================] - 0s 87us/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.0162 -

Epoch 21/80
3052/3052 [==============================] - 0s 82us/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.0130 - val_accuracy: 0.9971
Epoch 22/80
3052/3052 [==============================] - 0s 85us/step - loss: 0.0018 - accuracy: 0.9997 - val_loss: 0.0127 - val_accuracy: 0.9971
Epoch 23/80
3052/3052 [==============================] - 0s 89us/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.0147 - val_accuracy: 0.9971
Epoch 24/80
3052/3052 [==============================] - 0s 85us/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.0113 - val_accuracy: 0.9971
Epoch 25/80
3052/3052 [==============================] - 0s 87us/step - loss: 9.5184e-04 - accuracy: 1.0000 - val_loss: 0.0133 - val_accuracy: 0.9971
Epoch 26/80
3052/3052 [==============================] - 0s 86us/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.0128 - val_accuracy: 0.9941
Epoch 27/80
3052/3052 [==============================] - 0s 87us/step - loss: 8.6408e-04 - accuracy: 1.0000 - val_loss: 

In [11]:
# ==== CNN Removal =====
for n in range(NUM_region):
    region=region_image.copy()   #reset
    selected_region = list(range(NUM_region))
    selected_region.pop(n)

    print("n=", n)


    # ===== one CNN =============
    NUM_CLASSES = len(selected_region)

    # input image and label
    Input_img     = []
    Input_img_len = []
    for c,sel in enumerate(selected_region, start=0):
        Input_img = Input_img + list(region[sel])
        Input_img_len.append(len(region[sel]))

    # 20240319
    if RAW_2D_DATA: # 2D
        W           = np.shape(test_array[0])[0]
        H           = np.shape(test_array[0])[1]
        train_array = np.zeros((len(Input_img), W, H), dtype=float)
        for i in range (len(Input_img)):
            train_array[i] = test_array[Input_img[i]].reshape(W,H)
    else: # 1D
        W           = np.shape(test_array[0])[0]
        train_array = np.zeros((len(Input_img), W), dtype=float)
        for i in range (len(Input_img)):
            train_array[i] = test_array[Input_img[i]].reshape(W)
            
    train_array = np.expand_dims(train_array, axis = -1)
    
    
    # fill up the training label to each training image
    current_train_label = np.zeros(len(train_array), dtype=int)
    accum_base=0
    for label in range(1,NUM_CLASSES):
        pre_cursor = Input_img_len[label-1]
        accum_base = accum_base + pre_cursor
        current_train_label[accum_base:] = label

    # CNN
    #===============================================
    one_predicted_results  = np.zeros((TRIALS, len(test_label_answer)), dtype=int)
    one_predict_percentage = np.zeros((TRIALS, len(test_label_answer), NUM_CLASSES), dtype=float)    
    model_history = np.zeros(TRIALS, dtype=list)

    for r in range(TRIALS):
        one_predicted_results[r], one_predict_percentage[r], model_history[r] = ME_CNN(
                x_train     = train_array,
                train_label = current_train_label,
                test_array  = test_array,
                true_answer = test_label_answer,
                Num_Classes = NUM_CLASSES
                )
        print(type(model_history))


        # ===== delete CNN tensors =====
        from keras import backend as K
        K.clear_session()
        import gc
        gc.collect()

        print("One CNN, r: ",r)
        ROUND_duration = time.time() - ROUND_start
        print("Computing Time: ", str(datetime.timedelta(seconds=ROUND_duration)))

    # === save to file ===
    savefile_path = './' + str(timestr) + '(classes=' + str(NUM_CLASSES)+')_Remove' + str(n) +'.pickle' 
    with open(savefile_path, 'wb') as f:
        pickle.dump([Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history], f)


n= 0
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1000, 6)           36        
_________________________________________________________________
average_pooling1d_1 (Average (None, 500, 6)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 496, 16)           496       
_________________________________________________________________
average_pooling1d_2 (Average (None, 248, 16)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 244, 120)          9720      
_________________________________________________________________
flatten_1 (Flatten)          (None, 29280)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 84)          

3154/3154 [==============================] - 0s 88us/step - loss: 1.7928e-04 - accuracy: 1.0000 - val_loss: 0.0062 - val_accuracy: 0.9972
Epoch 47/80
3154/3154 [==============================] - 0s 86us/step - loss: 1.8173e-04 - accuracy: 1.0000 - val_loss: 0.0087 - val_accuracy: 0.9972
Epoch 48/80
3154/3154 [==============================] - 0s 83us/step - loss: 1.7951e-04 - accuracy: 1.0000 - val_loss: 0.0084 - val_accuracy: 0.9972
Epoch 49/80
3154/3154 [==============================] - 0s 94us/step - loss: 1.5689e-04 - accuracy: 1.0000 - val_loss: 0.0076 - val_accuracy: 0.9972
Epoch 50/80
3154/3154 [==============================] - 0s 88us/step - loss: 1.5014e-04 - accuracy: 1.0000 - val_loss: 0.0077 - val_accuracy: 0.9972
Epoch 51/80
3154/3154 [==============================] - 0s 86us/step - loss: 1.4127e-04 - accuracy: 1.0000 - val_loss: 0.0067 - val_accuracy: 0.9972
Epoch 00051: early stopping
[[2.5011564e-04 9.1440570e-01 2.0148400e-06 ... 1.2426640e-02
  1.7575091e-03 1.9644

Epoch 36/80
2983/2983 [==============================] - 0s 94us/step - loss: 6.4508e-04 - accuracy: 1.0000 - val_loss: 0.0014 - val_accuracy: 1.0000
Epoch 37/80
2983/2983 [==============================] - 0s 91us/step - loss: 3.6258e-04 - accuracy: 1.0000 - val_loss: 0.0011 - val_accuracy: 1.0000
Epoch 38/80
2983/2983 [==============================] - 0s 86us/step - loss: 2.9448e-04 - accuracy: 1.0000 - val_loss: 0.0011 - val_accuracy: 1.0000
Epoch 39/80
2983/2983 [==============================] - 0s 82us/step - loss: 2.5870e-04 - accuracy: 1.0000 - val_loss: 9.6733e-04 - val_accuracy: 1.0000
Epoch 40/80
2983/2983 [==============================] - 0s 89us/step - loss: 2.2967e-04 - accuracy: 1.0000 - val_loss: 8.9413e-04 - val_accuracy: 1.0000
Epoch 41/80
2983/2983 [==============================] - 0s 88us/step - loss: 2.0751e-04 - accuracy: 1.0000 - val_loss: 8.8962e-04 - val_accuracy: 1.0000
Epoch 42/80
2983/2983 [==============================] - 0s 88us/step - loss: 1.8958e-04

Train on 3091 samples, validate on 344 samples
Epoch 1/80
3091/3091 [==============================] - 1s 176us/step - loss: 2.4606 - accuracy: 0.1601 - val_loss: 2.4252 - val_accuracy: 0.2878
Epoch 2/80
3091/3091 [==============================] - 0s 106us/step - loss: 2.2814 - accuracy: 0.2866 - val_loss: 2.0272 - val_accuracy: 0.4477
Epoch 3/80
3091/3091 [==============================] - 0s 91us/step - loss: 1.4917 - accuracy: 0.6069 - val_loss: 0.9062 - val_accuracy: 0.7326
Epoch 4/80
3091/3091 [==============================] - 0s 85us/step - loss: 0.4454 - accuracy: 0.8900 - val_loss: 0.1934 - val_accuracy: 0.9593
Epoch 5/80
3091/3091 [==============================] - 0s 91us/step - loss: 0.1239 - accuracy: 0.9744 - val_loss: 0.0731 - val_accuracy: 0.9884
Epoch 6/80
3091/3091 [==============================] - 0s 92us/step - loss: 0.0556 - accuracy: 0.9884 - val_loss: 0.0387 - val_accuracy: 0.9913
Epoch 7/80
3091/3091 [==============================] - 0s 88us/step - loss: 0.03

[7 7 7 ... 7 3 7]
<class 'numpy.ndarray'>
One CNN, r:  0
Computing Time:  0:35:12.050241
n= 3
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1000, 6)           36        
_________________________________________________________________
average_pooling1d_1 (Average (None, 500, 6)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 496, 16)           496       
_________________________________________________________________
average_pooling1d_2 (Average (None, 248, 16)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 244, 120)          9720      
_________________________________________________________________
flatten_1 (Flatten)          (None, 29280)             0         
__________________________

...

3081/3081 [==============================] - 0s 87us/step - loss: 2.1218e-04 - accuracy: 1.0000 - val_loss: 8.2958e-04 - val_accuracy: 1.0000
Epoch 61/80
3081/3081 [==============================] - 0s 86us/step - loss: 2.1379e-04 - accuracy: 1.0000 - val_loss: 4.1644e-04 - val_accuracy: 1.0000
Epoch 62/80
3081/3081 [==============================] - 0s 93us/step - loss: 1.4174e-04 - accuracy: 1.0000 - val_loss: 4.4150e-04 - val_accuracy: 1.0000
Epoch 63/80
3081/3081 [==============================] - 0s 89us/step - loss: 1.0769e-04 - accuracy: 1.0000 - val_loss: 5.9287e-04 - val_accuracy: 1.0000
Epoch 64/80
3081/3081 [==============================] - 0s 87us/step - loss: 1.0458e-04 - accuracy: 1.0000 - val_loss: 6.8452e-04 - val_accuracy: 1.0000
Epoch 65/80
3081/3081 [==============================] - 0s 87us/step - loss: 9.3847e-05 - accuracy: 1.0000 - val_loss: 4.1322e-04 - val_accuracy: 1.0000
Epoch 66/80
3081/3081 [==============================] - 0s 90us/step - loss: 1.0221e-04

2790/2790 [==============================] - 0s 91us/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.0035 - val_accuracy: 1.0000
Epoch 21/80
2790/2790 [==============================] - 0s 86us/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.0044 - val_accuracy: 0.9968
Epoch 22/80
2790/2790 [==============================] - 0s 85us/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.0045 - val_accuracy: 0.9968
Epoch 23/80
2790/2790 [==============================] - 0s 90us/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.0035 - val_accuracy: 1.0000
Epoch 24/80
2790/2790 [==============================] - 0s 84us/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0043 - val_accuracy: 0.9968
Epoch 25/80
2790/2790 [==============================] - 0s 90us/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.0042 - val_accuracy: 0.9968
Epoch 26/80
2790/2790 [==============================] - 0s 90us/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.0022 - val_accurac

3076/3076 [==============================] - 0s 85us/step - loss: 0.0263 - accuracy: 0.9961 - val_loss: 0.0197 - val_accuracy: 0.9942
Epoch 9/80
3076/3076 [==============================] - 0s 87us/step - loss: 0.0106 - accuracy: 0.9990 - val_loss: 0.0131 - val_accuracy: 0.9971
Epoch 10/80
3076/3076 [==============================] - 0s 87us/step - loss: 0.0081 - accuracy: 0.9990 - val_loss: 0.0139 - val_accuracy: 0.9971
Epoch 11/80
3076/3076 [==============================] - 0s 83us/step - loss: 0.0085 - accuracy: 0.9990 - val_loss: 0.0107 - val_accuracy: 0.9971
Epoch 12/80
3076/3076 [==============================] - 0s 87us/step - loss: 0.0054 - accuracy: 0.9997 - val_loss: 0.0121 - val_accuracy: 0.9971
Epoch 13/80
3076/3076 [==============================] - 0s 92us/step - loss: 0.0045 - accuracy: 0.9997 - val_loss: 0.0090 - val_accuracy: 0.9971
Epoch 14/80
3076/3076 [==============================] - 0s 84us/step - loss: 0.0040 - accuracy: 0.9997 - val_loss: 0.0086 - val_accuracy

3076/3076 [==============================] - 0s 87us/step - loss: 1.4563e-04 - accuracy: 1.0000 - val_loss: 0.0027 - val_accuracy: 0.9971
Epoch 64/80
3076/3076 [==============================] - 0s 85us/step - loss: 1.3776e-04 - accuracy: 1.0000 - val_loss: 0.0026 - val_accuracy: 0.9971
Epoch 65/80
3076/3076 [==============================] - 0s 85us/step - loss: 1.3732e-04 - accuracy: 1.0000 - val_loss: 0.0025 - val_accuracy: 0.9971
Epoch 66/80
3076/3076 [==============================] - 0s 83us/step - loss: 1.3594e-04 - accuracy: 1.0000 - val_loss: 0.0025 - val_accuracy: 0.9971
Epoch 67/80
3076/3076 [==============================] - 0s 90us/step - loss: 1.2773e-04 - accuracy: 1.0000 - val_loss: 0.0025 - val_accuracy: 0.9971
Epoch 68/80
3076/3076 [==============================] - 0s 87us/step - loss: 1.3377e-04 - accuracy: 1.0000 - val_loss: 0.0024 - val_accuracy: 0.9971
Epoch 69/80
3076/3076 [==============================] - 0s 89us/step - loss: 1.2368e-04 - accuracy: 1.0000 - va

Epoch 24/80
3052/3052 [==============================] - 0s 86us/step - loss: 8.4917e-04 - accuracy: 1.0000 - val_loss: 0.0105 - val_accuracy: 0.9971
Epoch 25/80
3052/3052 [==============================] - 0s 86us/step - loss: 7.3413e-04 - accuracy: 1.0000 - val_loss: 0.0141 - val_accuracy: 0.9912
Epoch 26/80
3052/3052 [==============================] - 0s 88us/step - loss: 7.2325e-04 - accuracy: 1.0000 - val_loss: 0.0127 - val_accuracy: 0.9941
Epoch 27/80
3052/3052 [==============================] - 0s 86us/step - loss: 6.9215e-04 - accuracy: 1.0000 - val_loss: 0.0100 - val_accuracy: 0.9971
Epoch 28/80
3052/3052 [==============================] - 0s 85us/step - loss: 5.6853e-04 - accuracy: 1.0000 - val_loss: 0.0132 - val_accuracy: 0.9941
Epoch 29/80
3052/3052 [==============================] - 0s 87us/step - loss: 5.3795e-04 - accuracy: 1.0000 - val_loss: 0.0112 - val_accuracy: 0.9941
Epoch 30/80
3052/3052 [==============================] - 0s 88us/step - loss: 5.2686e-04 - accuracy:

In [12]:
# ====shift label=====
#=== combination =====
comb=[]
for oneset in itertools.combinations(range(NUM_region), 2):
    comb.append(oneset)
NUM_comb=len(comb)

Merged_result=[]
Merged_prob=[]
Merged_prob_label=[]

for n in range(NUM_comb):
    label = list(range(NUM_region))
    p1=comb[n][0]
    p2=comb[n][1]

    with open('./' + timestr + '(classes=' + str(NUM_region-1) + ')_n'+str(n+1)+'_R'+str(p1)+'+R'+str(p2)+'.pickle', 'rb') as f:
        Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history = pickle.load(f)
    label_B = one_predicted_results[0].copy()
    label_B_prob = one_predict_percentage[0].copy()

    #(1)shift label index
    for p in reversed(range(p2,NUM_region-1)):
        addr=np.where(label_B==p)[0]
        label_B[addr]+=1
    Merged_result.append(label_B)

    #(2)
    Merged_prob.append(label_B_prob)

    #(3)
    label.pop(p2)
    Merged_prob_label.append(label)

print(np.shape(Merged_result), np.shape(Merged_prob), np.shape(Merged_prob_label))

# save pickle
with open('./' + timestr + 'results_of_combination.pickle', 'wb') as f:
    pickle.dump([comb, Merged_result, Merged_prob, Merged_prob_label], f)

# save mat
from scipy.io import savemat
if (np.shape(Merged_prob)[0]<=300):
    savemat('./' + timestr + 'results_of_combination.mat', {'combination_pairs':comb, 'result_for_merge':Merged_result, 'prob_for_merge':Merged_prob,'prob_label_for_merge': Merged_prob_label})
    print("normal size = ", np.shape(Merged_prob))
else:
    savemat('./' + timestr + 'results_of_combination.mat', {'combination_pairs':comb, 'result_for_merge':Merged_result, 'prob_label_for_merge': Merged_prob_label})
    savemat('./' + timestr + 'results_of_combination_prob1.mat', {'prob_for_merge1':Merged_prob[:200]})
    savemat('./' + timestr + 'results_of_combination_prob2.mat', {'prob_for_merge2':Merged_prob[200:400]})
    savemat('./' + timestr + 'results_of_combination_prob3.mat', {'prob_for_merge3':Merged_prob[400:]})
    print("large size = ", np.shape(Merged_prob))

    
    
    
#======== removal ===========
Removal_result=[]
Removal_prob=[]
Removal_prob_label=[]

for n in range(NUM_region):
    label = list(range(NUM_region))    
    #reset
    with open('./' + timestr + '(classes=' + str(NUM_region-1) + ')_Remove' + str(n) + '.pickle', 'rb') as f:
        Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history = pickle.load(f)
    label_B = one_predicted_results[0].copy()
    label_B_prob = one_predict_percentage[0].copy()

    for p in reversed(range(n,NUM_region-1)):
        addr=np.where(label_B==p)[0]
        label_B[addr]+=1
    Removal_result.append(label_B)

    #(2)
    Removal_prob.append(label_B_prob)

    #(3)
    label.pop(n)
    Removal_prob_label.append(label)

print(np.shape(Removal_result), np.shape(Removal_prob), np.shape(Removal_prob_label))

# save pickle
with open('./' + timestr + 'results_of_removal.pickle', 'wb') as f:
    pickle.dump([Removal_result, Removal_prob, Removal_prob_label], f)
    
# save mat
from scipy.io import savemat
savemat('./' + timestr + 'results_of_removal.mat', {'result_for_removal':Removal_result,'prob_for_removal':Removal_prob, 'prob_label_for_removal':Removal_prob_label})

(91, 13076) (91, 13076, 13) (91, 13)
normal size =  (91, 13076, 13)
(14, 13076) (14, 13076, 13) (14, 13)


In [13]:
print("done")

done
